In [103]:
import numpy as np
import itertools
import random
import simpy

In [104]:
# All units are in minutes

WEEKS = 2

# Simulation time in minutes
SIM_TIME = 168 * WEEKS * 60              

# number of docks/berth
DOCKS = 2

# number of tug/crane 
TUGS = 4

# shape parameter of Erlang
K = 3

UNLOAD_SPEED = 1
UNLOADING_TIME = 120.0

# average inter-arrival times for ships of type 1 to 3
AVG_TRAVEL_TYPE1_3 = 180.0

# average unloading time for ships of type 1
UNLOAD_TIME_TYPE_1 = UNLOADING_TIME

# average unloading time for ships of type 2
UNLOAD_TIME_TYPE_2 = UNLOADING_TIME

# average unloading time for ships of type 3
UNLOAD_TIME_TYPE_3 = UNLOADING_TIME

BERTHING_TIME = 45.0
DEBERTHING_TIME = 30.0


"""----- Ship distribution by type ----"""
# ship distribution of ships type 1 to 3 {type 1: 25%, type 2: 55%, type 3: 20%}
SHIP_DISTR = [1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3]

"""-----------LOG----------------"""
# wait crane to berthing
crane_wt1 = []
# wait for berth
berth_wt = []
# wait crane to deberthing
crane_wt2 = []

# help to analyze results
ship_departure_times = []

In [105]:
def ship_generator(env, tug, dock):
    #Generate new ships.
    i = 0
    while True:
        yield env.timeout(1)
        # randomly select a ship type from SHIP_DISTR
        shiptype = random.choice(SHIP_DISTR)
        env.process(ship('Ship %d' % i, env, tug, dock, shiptype))
        yield env.timeout(time_to_arrive())
        i += 1

def time_to_arrive():
    # Erlang/Gamma distribution with shape k, scale theta
    theta = AVG_TRAVEL_TYPE1_3/K
    return random.gammavariate(K,theta)
    
def time_to_berth():
    return random.expovariate(1.0/BERTHING_TIME)

def time_to_deberth():
    return random.expovariate(1.0/DEBERTHING_TIME)

def time_to_unload(shiptype):
    unloadTime = {
            1 : UNLOAD_TIME_TYPE_1,
            2 : UNLOAD_TIME_TYPE_2,
            3 : UNLOAD_TIME_TYPE_3,
                  }
    return random.expovariate(1.0/unloadTime[shiptype])

def ship(name, env, tug, dock, shiptype):
    print('%s arriving at port at %.1f' % (name, env.now))
    with tug.request() as req:
        # Request the tug
        wt = env.now
        yield req
        wt = env.now - wt
        crane_wt1.append(wt)
        
        # request for dock and process ship at dock
        env.process(shipAtDock(name, env, dock, shiptype))

        
def tug_transport(name, env, tug, shiptype):
    """Arrives at the port or the dock after a 1h delay and transports the ship."""
    # print('Tug starts %s transport at %.1f' % (name, env.now))
    yield env.timeout(TUG_TIME_ONE_WAY)
    # print('Tug stops %s transport at %.1f' % (name, env.now))


def shipAtDock(name, env, dock, shiptype):
    with dock.request() as req:
        # Request a dock
        wt = env.now
        yield req
        wt = env.now - wt
        berth_wt.append(wt)
        
        # Berthing
        yield env.timeout(time_to_berth())
        
        print('%s of shiptype %d unloading at %.1f' % (name, shiptype, env.now))
        yield env.timeout(time_to_unload(shiptype))
        print('%s unloaded at %.1f' % (name, env.now))
        
        with tug.request() as req:
            # Request the tug
            wt = env.now
            yield req
            wt = env.now - wt
            crane_wt2.append(wt)

            # Deberthing
            yield env.timeout(time_to_deberth())
            
            ship_departure_times.append(env.now)
            print('%s departed at %.1f' % (name, env.now))

In [106]:
# Setup and start the simulation
print('Marine Transport Simulation')

# Create environment and start processes
env = simpy.Environment()

# ships_type_4 = simpy.Resource(env, 1)

# Resource with capacity of usage slots that can be requested by processes.
tug = simpy.Resource(env, TUGS)

# Resource with capacity of usage slots that can be requested by processes.
dock = simpy.Resource(env, DOCKS)

# Process an event yielding generator.
env.process(ship_generator(env, tug, dock))

# Execute!
env.run(until=SIM_TIME)

Marine Transport Simulation
Ship 0 arriving at port at 1.0
Ship 0 of shiptype 2 unloading at 10.4
Ship 0 unloaded at 113.4
Ship 0 departed at 126.4
Ship 1 arriving at port at 401.1
Ship 1 of shiptype 2 unloading at 408.3
Ship 2 arriving at port at 480.7
Ship 2 of shiptype 1 unloading at 501.7
Ship 2 unloaded at 608.3
Ship 2 departed at 618.7
Ship 1 unloaded at 646.4
Ship 1 departed at 667.2
Ship 3 arriving at port at 806.8
Ship 3 of shiptype 2 unloading at 832.9
Ship 4 arriving at port at 853.4
Ship 4 of shiptype 2 unloading at 883.9
Ship 5 arriving at port at 919.8
Ship 3 unloaded at 951.3
Ship 4 unloaded at 1042.5
Ship 6 arriving at port at 1045.3
Ship 3 departed at 1055.1
Ship 4 departed at 1073.2
Ship 6 of shiptype 2 unloading at 1074.2
Ship 5 of shiptype 2 unloading at 1083.3
Ship 5 unloaded at 1115.9
Ship 5 departed at 1125.3
Ship 7 arriving at port at 1229.0
Ship 6 unloaded at 1236.0
Ship 7 of shiptype 1 unloading at 1266.6
Ship 6 departed at 1272.5
Ship 7 unloaded at 1324.3
Shi

In [110]:
num_ship = min(len(crane_wt1), len(crane_wt2), len(berth_wt))
ship_departure_times = ship_departure_times[0:num_ship-1]
ship_delay_times = np.add(berth_wt[0:num_ship-1], np.add(crane_wt1[0:num_ship-1], crane_wt2[0:num_ship-1]))

In [111]:
print ship_departure_times
print ship_delay_times

[126.35730320198535, 618.7209929138295, 667.1782755580854, 1055.0597236825533, 1073.1994162594108, 1125.3080963469736, 1272.461143807076, 1384.403598277326, 1460.9653895982228, 1854.18744681117, 2068.444335616572, 2175.4171564558997, 2339.429333878295, 2616.556328855228, 2736.389291726757, 2973.745648904091, 3147.8418781967107, 3420.6843554455904, 3524.2047999654355, 3659.8433955756195, 3674.283228817199, 3677.7942644144587, 4411.199476267047, 4474.056432125801, 4476.936460388588, 4603.020166406667, 4790.010116146657, 5131.786620386297, 5151.089500160474, 5599.017461732021, 5749.294673087302, 5907.502450146182, 6006.319608373708, 6147.656430930317, 6286.792458274835, 6500.560102423789, 6734.744500237391, 6914.8638575392815, 7197.903136393983, 7262.50651639083, 7483.343766496611, 7647.707564995686, 7859.479081540635, 8569.01039097458, 8660.12843040981, 8881.633863382098, 8959.462128275367, 9138.192855050274, 9338.696719185631, 9648.76418392018, 9951.486951764224, 10054.684579024239, 101